In [5]:
!pip install --upgrade pip
!pip install llama-index==0.9.41 langchain==0.3.27 langchain_community openai==1.101.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.5 MB/s  0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.74
    Uninstalling langchain-core-0.3.74:
      Successfully uninstalled langchain-core-0.3.74
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [langchain_community]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [1]:
from llama_index import SimpleDirectoryReader, GPTVectorStoreIndex, LLMPredictor, ServiceContext
from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool, initialize_agent, AgentType
import os


In [4]:
os.environ["OPENAI_API_KEY"] = "openai_api_key"

In [5]:

from google.colab import files

os.makedirs("docs", exist_ok=True)

uploaded = files.upload()

for filename in uploaded.keys():
    os.rename(filename, f"docs/{filename}")

print("Uploaded files:", os.listdir("docs"))


Saving LLM_info.txt.txt to LLM_info.txt.txt
Saving RAG_notes.txt.txt to RAG_notes.txt.txt
Uploaded files: ['LLM_info.txt.txt', 'RAG_notes.txt.txt']


In [12]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'wikipedia' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'wikipedia'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=6874ede91e3b917ce7a79916deba78d24322f5777a89b9944761facc1e855cc1
  Stored in directory: /root/.cache/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia


In [13]:
from llama_index import SimpleDirectoryReader, GPTVectorStoreIndex, LLMPredictor, ServiceContext
from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferMemory
import wikipedia


In [14]:
# Load documents
documents = SimpleDirectoryReader("docs/").load_data()

# Initialize LLM predictor
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))

# Service context
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

# Vector store index (top_k retrieval)
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)
query_engine = index.as_query_engine(retriever_mode="default", similarity_top_k=3)


LLMPredictor is deprecated, please use LLM instead.


In [15]:
# Document retrieval tool
def retrieve_docs(query: str) -> str:
    response = query_engine.query(query)
    return str(response)

# Simple calculator tool
def calculator(query: str) -> str:
    try:
        return str(eval(query))
    except:
        return "Cannot calculate that."

# Wikipedia search tool
def wiki_search(query: str) -> str:
    try:
        summary = wikipedia.summary(query, sentences=2)
        return summary
    except:
        return "No Wikipedia info found."


In [16]:
tools = [
    Tool(name="DocumentRetriever", func=retrieve_docs, description="Use this tool to answer questions using uploaded documents."),
    Tool(name="Calculator", func=calculator, description="Use this tool to solve math expressions."),
    Tool(name="Wikipedia", func=wiki_search, description="Use this tool to get Wikipedia summaries.")
]


In [17]:
# Conversation memory
memory = ConversationBufferMemory(memory_key="chat_history")

agent = initialize_agent(
    tools=tools,
    llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"),
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)


/tmp/ipython-input-724016799.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")


In [19]:
print("Agentic RAG is ready! Type 'exit' to stop.")

while True:
    query = input("You: ")
    if query.lower() in ["exit", "quit"]:
        break
    response = agent.run(query)
    print("Agent:", response)


Agentic RAG is ready! Type 'exit' to stop.
You: What is RAG?


> Entering new AgentExecutor chain...
I should use Wikipedia to find the definition of RAG.
Action: Wikipedia
Action Input: RAG
Observation: A bag, also known regionally as a sack, is a common tool in the form of a floppy container, typically made of cloth, leather, bamboo, paper, or plastic. The use of bags predates recorded history, with the earliest bags being lengths of animal skin, cotton, or woven plant fibers, folded up at the edges and secured in that shape with strings of the same material.
Thought:That doesn't seem to be the correct definition of RAG. I should try a different approach.
Action: DocumentRetriever
Action Input: RAG
Observation: RAG stands for Retrieval-Augmented Generation, which is an AI technique that enhances the accuracy and relevance of responses by combining large language models with information retrieval systems. It addresses limitations of solely relying on pre-trained data by fetching real-

/usr/local/lib/python3.12/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.12/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: No Wikipedia info found.
Thought:I should try using the DocumentRetriever tool to find information about LLM.
Action: DocumentRetriever
Action Input: LLM
Observation: LLMs are large language models trained on massive text datasets. They can generate, summarize, or answer questions based on context.
Thought:I now know the information about LLM.
Final Answer: LLMs are large language models trained on massive text datasets. They can generate, summarize, or answer questions based on context.

> Finished chain.
Agent: LLMs are large language models trained on massive text datasets. They can generate, summarize, or answer questions based on context.
You: What is 27864 multipied by 48724


> Entering new AgentExecutor chain...
I should use the Calculator tool to multiply these two numbers.
Action: Calculator
Action Input: 27864 * 48724
Observation: 1357645536
Thought:I now know the final answer
Final Answer: 1357645536

> Finished chain.
Agent: 1357645536
You: Exit
